<a href="https://colab.research.google.com/github/tauseefhashmi/TrafficSignClassification/blob/main/Analytics_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install keras
# !pip install tensorflow
# !pip install pandas 
# !pip install sklearn 
# !pip install matplotlib 
!pip install -U efficientnet


In [15]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
from zipfile import *
import efficientnet.keras as efn
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [ ]:
!unzip -q /content/drive/MyDrive/GoogleCollabWork/Traffic.zip -d /content/drive/MyDrive/GoogleCollabWork/Traffic1  #unzip data in tmp/

In [3]:
base_dir = '/content/drive/MyDrive/GoogleCollabWork/Traffic1'
train_dir = '/content/drive/MyDrive/GoogleCollabWork/Traffic1/Train'
validation_dir ='/content/drive/MyDrive/GoogleCollabWork/Traffic1/Test'


In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
import matplotlib.image as mpimg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100
train_traffic_fnames = os.listdir(train_dir)

next_cat_pix = [os.path.join(train_dir, fname) 
                for fname in train_traffic_fnames[ pic_index-8:pic_index] 
               ]

for i, img_path in enumerate(next_cat_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [5]:
# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory( validation_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 39209 images belonging to 43 classes.
Found 0 images belonging to 0 classes.


In [6]:
import efficientnet.keras as efn

In [9]:
from tensorflow.keras.applications import EfficientNetB0
model = EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

16711680/16705208 [==============================] - 0s 0us/step


In [13]:
for layer in model.layers:
    layer.trainable = False

In [18]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = layers.Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(model.input, predictions)

In [21]:
model_final.compile(tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
eff_history = model_final.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
100/100 [==============================] - 462s 4s/step - loss: -23276.0449 - accuracy: 0.0595
Epoch 2/10
100/100 [==============================] - 363s 4s/step - loss: -79866.7266 - accuracy: 0.0570
Epoch 3/10
100/100 [==============================] - 337s 3s/step - loss: -159211.5312 - accuracy: 0.0555
Epoch 4/10
100/100 [==============================] - 320s 3s/step - loss: -247016.2344 - accuracy: 0.0525
Epoch 5/10
100/100 [==============================] - 317s 3s/step - loss: -361018.5000 - accuracy: 0.0498
Epoch 6/10
100/100 [==============================] - 303s 3s/step - loss: -491751.2500 - accuracy: 0.0555
Epoch 7/10
100/100 [==============================] - 283s 3s/step - loss: -631956.8750 - accuracy: 0.0590
Epoch 8/10
100/100 [==============================] - 267s 3s/step - loss: -805892.7500 - accuracy: 0.0533
Epoch 9/10
100/100 [==============================] - 254s 3s/step - loss: -975024.5000 - accuracy: 0.0545
Epoch 10/10
 86/100 [==================

#VGG

#ResNet

#FLASK DEPLOYMENT

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def home():
    return "<h1>GFG is great platform to learn</h1>"
    
app.run()